In [ ]:
import numpy as np, pandas as pd, gc

import matplotlib.pyplot as plt

import os
import cv2

from tqdm.notebook import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
!pip install timm
import timm

In [ ]:
train_csv = pd.read_csv('../input/shopee-product-matching/train.csv')
train_csv

In [ ]:
train_csv.isnull().sum()

In [ ]:
tmp = train_csv.groupby('label_group').posting_id.agg('unique').to_dict()
train_csv['target'] = train_csv.label_group.map(tmp)
train_csv.head()

In [ ]:
dataset_dir = "../input/shopee-product-matching/"

train_images = dataset_dir + "/train_images/" + train_csv['image']
train_csv['path'] = train_images

In [ ]:
BASE = '../input/shopee-product-matching/train_images/'

def displayDF(train_csv, random=False, COLS=6, ROWS=4, path=BASE):
    for k in range(ROWS):
        plt.figure(figsize=(20,5))
        for j in range(COLS):
            if random: row = np.random.randint(0,len(train_csv))
            else: row = COLS*k + j
            name = train_csv.iloc[row,1]
            title = train_csv.iloc[row,3]
            title_with_return = ""
            for i,ch in enumerate(title):
                title_with_return += ch
                if (i!=0)&(i%20==0): title_with_return += '\n'
            img = cv2.imread(path+name)
            plt.subplot(1,COLS,j+1)
            plt.title(title_with_return)
            plt.axis('off')
            plt.imshow(img)
            plt.show()
        
displayDF(train_csv,random=True)

In [ ]:
groups = train_csv.label_group.value_counts()
for k in range(5):
    print('#'*40)
    print('### TOP %i DUPLICATED ITEM:'%(k+1),groups.index[k])
    print('#'*40)
    top = train_csv.loc[train_csv.label_group==groups.index[k]]
    displayDF(top, random=False, ROWS=2, COLS=4)

> Since images are difference than the others, images aren't duplicated.

In [ ]:
cols = ['posting_id', 'target']
submit = train_csv[cols]
submit.to_csv('submission.csv', index = False)

In [ ]:
submit.head()